In [19]:
import pandas as pd
import random
from pytube import YouTube
from moviepy.editor import *

In [21]:
print(moviepy.editor__version__)

NameError: name 'moviepy' is not defined

In [2]:
audio_options = pd.read_csv("NoCopyrightSounds - WithDescriptions.csv")
selected_audio = audio_options.sample()
audio_url = selected_audio['videoURL'].iloc[0]

In [3]:
vids = [
    "https://www.youtube.com/watch?v=XYLg6Y1eMzo",
    "https://www.youtube.com/watch?v=muczNvx9fgg",
    "https://www.youtube.com/watch?v=cEHP_LeBeyQ",
    "https://www.youtube.com/watch?v=iUtnZpzkbG8",
    "https://www.youtube.com/watch?v=TfuNwm3D3Qc",
    "https://www.youtube.com/watch?v=Sq9eRhmnghE",
    "https://www.youtube.com/watch?v=xUZOXrb1c4E",
    "https://www.youtube.com/watch?v=KaHypeOcui0",
    "https://www.youtube.com/watch?v=vhpOhHEhVOg"
]

In [4]:
vid_length = []
video_yt = []
vid_time = 0

for vid in vids:
    while vid_time == 0:
        vid_object = YouTube(vid)
        try:
            vid_time = vid_object.length
        except:
            vid_time = 0
        
    vid_length.append(vid_time)
    video_yt.append(vid_object)
    vid_time = 0 
        
vid_length = pd.DataFrame({'video_length':vid_length})

In [5]:
vid_data = pd.DataFrame({'video_url':vids, 'video_yt':video_yt})
vid_data['video_id'] = vid_data['video_url'].str.extract(r'v=(\w+)')
vid_data = pd.concat([vid_data, vid_length], axis=1)

In [6]:
vid_data

,video_url,video_yt,video_id,video_length
0,https://www.youtube.com/watch?v=XYLg6Y1eMzo,<pytube.__main__.YouTube object: videoId=XYLg6...,XYLg6Y1eMzo,515
1,https://www.youtube.com/watch?v=muczNvx9fgg,<pytube.__main__.YouTube object: videoId=muczN...,muczNvx9fgg,677
2,https://www.youtube.com/watch?v=cEHP_LeBeyQ,<pytube.__main__.YouTube object: videoId=cEHP_...,cEHP_LeBeyQ,368
3,https://www.youtube.com/watch?v=iUtnZpzkbG8,<pytube.__main__.YouTube object: videoId=iUtnZ...,iUtnZpzkbG8,185
4,https://www.youtube.com/watch?v=TfuNwm3D3Qc,<pytube.__main__.YouTube object: videoId=TfuNw...,TfuNwm3D3Qc,603
5,https://www.youtube.com/watch?v=Sq9eRhmnghE,<pytube.__main__.YouTube object: videoId=Sq9eR...,Sq9eRhmnghE,659
6,https://www.youtube.com/watch?v=xUZOXrb1c4E,<pytube.__main__.YouTube object: videoId=xUZOX...,xUZOXrb1c4E,1327
7,https://www.youtube.com/watch?v=KaHypeOcui0,<pytube.__main__.YouTube object: videoId=KaHyp...,KaHypeOcui0,491
8,https://www.youtube.com/watch?v=vhpOhHEhVOg,<pytube.__main__.YouTube object: videoId=vhpOh...,vhpOhHEhVOg,609


In [7]:
audio_length = 0
while audio_length == 0:
        audio_yt = YouTube(audio_url)
        try:
            audio_length = audio_yt.length
        except:
            audio_length = 0

In [8]:
video_options = vid_data[vid_data["video_length"] > audio_length] 

In [9]:
selected_video = video_options.sample()['video_yt'].iloc[0]

In [10]:
selected_video.watch_url

'https://youtube.com/watch?v=Sq9eRhmnghE'

In [11]:
audio_yt.watch_url

'https://youtube.com/watch?v=D-TQB-T-UJ4'

In [12]:
audio_yt.streams.filter(only_audio = True, file_extension = 'mp4').first().download(filename="audio.mp3")
selected_video.streams.filter(only_video = True, file_extension = 'mp4').first().download(filename = "video_no_audio.mp4")

'/Users/user/Documents/GitHub/TikTokSideHustle/video_no_audio.mp4'

In [13]:
video_clip = VideoFileClip("video_no_audio.mp4")
audio_clip = AudioFileClip("audio.mp3")
video_clip = video_clip.subclip(0, audio_length)

new_audioclip = CompositeAudioClip([audio_clip])
video_clip.audio = new_audioclip
video_clip.write_videofile("video.mp4")

Moviepy - Building video video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4
